In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('30ktweet.csv')

In [4]:
df.columns=['tweets','sentiments']

In [5]:
df

,tweets,sentiments
0,@robbiebronniman Sounds like a great night.,1
1,Damn the person who stolde my wallet !!!!! Ma...,1
2,Greetings from the piano bench (photo) http:/...,1
3,@drewryanscott i love it!! i love you!! haha f...,1
4,"@kissthestars Pretty pretty pretty please, pak...",0
...,...,...
29995,@Calumfan1 is it in any way related to photosh...,0
29996,@Swiz_NZ really? wow thats crap,0
29997,"At the 2010 lexus HS250h press event. Again, ...",0
29998,@karmicunderpath ooooh now there's a nice thou...,1


In [6]:
df['sentiments'].value_counts()


0    15000
1    15000
Name: sentiments, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
def svm(df):
    message=df['tweets']
    y=df['sentiments']

    tfidf=TfidfVectorizer()
    X=tfidf.fit_transform(message)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)
    
    clf=LinearSVC()
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    print(classification_report(y_test, y_pred))
    
    return tfidf, clf

In [9]:
%%time
tfidf, clf = svm(df)

              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3000
           1       0.74      0.75      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000

Wall time: 1.29 s


In [10]:
x=['Thank you for your help']

In [11]:
clf.predict(tfidf.transform(x))

array([1], dtype=int64)

In [12]:
import preprocess_vendroid as ven

In [13]:
df['tweets']=df['tweets'].apply(lambda x:x.lower())

In [14]:
df['tweets']=df['tweets'].apply(lambda x :  ven.cont_exp(x))

In [15]:
df

,tweets,sentiments
0,@robbiebronniman sounds like a great night.,1
1,damn the person who stolde my wallet !!!!! ma...,1
2,greetings from the piano bench (photo) http:/...,1
3,@drewryanscott i love it!! i love you!! haha f...,1
4,"@kissthestars pretty pretty pretty please, pak...",0
...,...,...
29995,@calumfan1 is it in any way related to photosh...,0
29996,@swiz_nz really? wow thats crap,0
29997,"at the 2010 lexus hs250h press event. again, ...",0
29998,@karmicunderpath ooooh now there's a nice thou...,1


In [16]:
svm(df)

              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3000
           1       0.74      0.75      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000



(TfidfVectorizer(), LinearSVC())

In [17]:
# remove emails and urls

df['tweets'] = df['tweets'].apply(lambda x: ven.remove_emails(x))
df['tweets'] = df['tweets'].apply(lambda x: ven.remove_urls(x))
df['tweets'] = df['tweets'].apply(lambda x: ven.remove_rt(x))

df['tweets'] = df['tweets'].apply(lambda x: ven.remove_special_chars(x))


In [18]:
svm(df)

              precision    recall  f1-score   support

           0       0.75      0.74      0.74      3000
           1       0.74      0.75      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000



(TfidfVectorizer(), LinearSVC())

In [19]:
def svm(df):
    message=df['tweets']
    y=df['sentiments']

    tfidf=TfidfVectorizer(norm = 'l1', ngram_range=(1,2), analyzer='word')
    X=tfidf.fit_transform(message)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)
    
    clf=LinearSVC()
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    print(classification_report(y_test, y_pred))
    
    return tfidf, clf

In [20]:
svm(df)

              precision    recall  f1-score   support

           0       0.76      0.76      0.76      3000
           1       0.76      0.76      0.76      3000

    accuracy                           0.76      6000
   macro avg       0.76      0.76      0.76      6000
weighted avg       0.76      0.76      0.76      6000



(TfidfVectorizer(ngram_range=(1, 2), norm='l1'), LinearSVC())

In [21]:
import pickle

In [24]:
pickle.dump(clf, open('clf.pkl', 'wb'))
pickle.dump(tfidf, open('tfidf.pkl', 'wb'))